### Read Videos & Convert to Frames

In [46]:
import os
import logging
import subprocess

In [47]:
def create_folders_and_export_videos(directory_path, image_folder_path):
    logging.basicConfig(level=logging.INFO) 

    # List all .svo video files in the specified directory
    svo_files = [file for file in os.listdir(directory_path) if file.lower().endswith('.svo')]

    # Process each .svo video file
    for svo_file in svo_files:
        video_path = os.path.join(directory_path, svo_file)
        output_path = os.path.join(image_folder_path, os.path.splitext(svo_file)[0])

        # Check if the folder already exists
        if not os.path.exists(output_path):
            os.makedirs(output_path)
            logging.info(f"Folder '{os.path.splitext(svo_file)[0]}' created at {output_path}")
        else:
            logging.info(f"Folder '{os.path.splitext(svo_file)[0]}' already exists at {output_path}")

        # Run ZED_SVO_Export.exe command for .svo files (must be run from the same directory as the SDK so change the directories first)
        zed_sdk_bin_path = 'C:/Program Files (x86)/ZED SDK/samples/bin'
        os.chdir(zed_sdk_bin_path)
        command = f'ZED_SVO_Export.exe "{video_path}" "{output_path}/" 3'
        subprocess.run(command, shell=True)

        # Check the exported files and delete those that do not start with "left" (we don't want the depth information)
        exported_files = [file for file in os.listdir(output_path) if file.lower().endswith('.png')]
        for exported_file in exported_files:
            if not exported_file.lower().startswith('left'):
                file_path = os.path.join(output_path, exported_file)
                os.remove(file_path)
                # logging.info(f"File '{exported_file}' deleted as it does not start with 'left'")

        print(f".svo file '{svo_file}' exported and saved in '{output_path}'")
        
    return output_path

### Convert to Grayscale

In [48]:
from matplotlib import pyplot as plt
import cv2
import numpy as np

In [49]:
def convertToGrayScale(path, output_directory):
    # List all frames of the video in the folder with the specified directory_path
    videoFrames = [frame for frame in os.listdir(path)]       # name of file

    # List all 
    grayImagesList = [] 
    for frame in videoFrames:
        frame_path = os.path.join(path, frame)
        grayImage = cv2.imread(frame_path, cv2.IMREAD_GRAYSCALE)
        grayImagesList.append(grayImage.astype(np.int16))
        
        # Save the grayscale image to a directory
        output_path = os.path.join(output_directory, f"gray_{frame}")
        cv2.imwrite(output_path, grayImage)
        
    print(f"video images '{output_directory[-6:]}' converted to grayscale and saved in '{output_directory}'")
    # Need to make sure all files are named the same to have correct printing

    return grayImagesList # might remove and reurn path instead

### Directional ATD

##### The process of ATD involves using thresholds to set insignificant differences of pixel values to 0. Insignificant differences represent minimal to no change in motion between two frames. For the difference of any two frames in a sequence, extreme or ‘noise’ values are set to 0 to reduce the effect on the average pixel value. For example, absolute values below 13 can be set to 0 and thoseabove 13 are kept as is. Afterwards, the average of the pixel values μ of the differenced frame is taken and used as the threshold for setting insignificant differences to 0 and significant ones to 1. This process, known as binarization, is performed for every pair of sequential frames. Once complete, the values of all differences are accumulated, resulting in one frame with pixel values indicating significant change in motion over a sequence of time.

In [50]:
from matplotlib import pyplot as plt 
import numpy as np

In [51]:
def performThreshold(diffFrame):
    mask = (diffFrame < 13) & (diffFrame > -13)
    diffFrame[mask] = 0
    return diffFrame

In [52]:
def binarize(noiseless_image):
    mean = np.mean(noiseless_image)
    maskZeroes = (noiseless_image < mean)
    maskOnes = (noiseless_image >= mean)
    binarizedImage = noiseless_image
    binarizedImage[maskZeroes] = 0
    binarizedImage[maskOnes] = 1
    return binarizedImage

In [53]:
def accumulateFrames(framesList):
    accumulatedFrame = framesList[0]

    for i in range(1, len(framesList)):
        accumulatedFrame += framesList[i]
    
    return accumulatedFrame

In [54]:
def applyForwardDiff(grayImagesList):
    listOfFrames = []

    for index in range(len(grayImagesList) - 1):
        forward_ImageDiff = grayImagesList[index] - grayImagesList[index + 1]
        forward_ImageDiff = performThreshold(forward_ImageDiff)
        binarizedImage = binarize(forward_ImageDiff)        
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [55]:
def applyBackwardDiff(grayImagesList):
    listOfFrames = []

    for index in range(1, len(grayImagesList)):
        backward_ImageDiff = grayImagesList[index] - grayImagesList[index - 1]
        backward_ImageDiff = performThreshold(backward_ImageDiff)
        binarizedImage = binarize(backward_ImageDiff) 
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [56]:
def applyBiDirectionalDiff(grayImagesList):
    listOfFrames = []

    for index in range(1, len(grayImagesList)-1):
        average_frame = np.mean([grayImagesList[index - 1], grayImagesList[index + 1]], axis=0)
        bidirectional_ImageDiff = grayImagesList[index] - average_frame
        bidirectional_ImageDiff = performThreshold(bidirectional_ImageDiff)
        binarizedImage = binarize(bidirectional_ImageDiff)
        listOfFrames.append(binarizedImage)
    
    return accumulateFrames(listOfFrames)

In [57]:
def applyATD(frames, output_directory, index):
    forwardATD = applyForwardDiff(frames)
    backwardATD = applyBackwardDiff(frames)
    bidirectATD = applyBiDirectionalDiff(frames)
    
    # Convert to np.uint8
    # forwardATD = forwardATD.astype(np.uint8)
    # backwardATD = backwardATD.astype(np.uint8)
    # bidirectATD = bidirectATD.astype(np.uint8)
    
    # Save the atd images (RGB) to a directory
    type(forwardATD)
    # image_ForwardATD_path = os.path.join(output_directory, f"forwardATD_word{index}")
    # plt.imsave(image_ForwardATD_path, forwardATD)
    
    # image_BackwardATD_path = os.path.join(output_directory, f"backwardATD_word{index}")
    # cv2.imwrite(image_BackwardATD_path, backwardATD)
    
    # image_BidirectionalATD_path = os.path.join(output_directory, f"bidirectionalATD_word{index}")
    # cv2.imwrite(image_BidirectionalATD_path, bidirectATD)
        
    # print(f"ATD frames '{output_directory[-6:]}_word{index}' created and saved in '{output_directory}'")
    # # Need to make sure all files are named the same to have correct printing

    # stackedImage = np.dstack(forwardATD, backwardATD, bidirectATD)#.astype(np.uint8)
    
    # stackedImage_path = os.path.join(output_directory, f"stackedImage_word{index}")
    # cv2.imwrite(stackedImage_path, stackedImage)
    
    # print(f"Stacked Image '{output_directory[-6:]}_word{index}' created and saved in '{output_directory}'")
    # # Need to make sure all files are named the same to have correct printing

### Main
Running Image Sequence then GrayScale then ATD then Stacking

In [58]:
## Main1: Creating External Files


base_directory = 'C:/Users/salma/Downloads/file' # CHANGE

## Image Sequence File ##
# Create 'file2' folder if not exists (the folder holding the frames of each video)
file2_path = os.path.join('C:/Users/salma/Downloads/file2')  # CHANGE
if not os.path.exists(file2_path):
    os.makedirs(file2_path)
    logging.info(f"Folder 'file2' created at {file2_path}")

## GrayScale File ##
# Create 'file3' folder if not exists (the folder holding greyscale image for each video)
file3_path = os.path.join('C:/Users/salma/Downloads/file3') # CHANGE
if not os.path.exists(file3_path):
    os.makedirs(file3_path)
    logging.info(f"Folder 'file3' created at {file3_path}")
    
## ATD and Stacked Image ##
# Create 'file4' folder if not exists (The folder holding the ATD Frams - RGB and Stacked image)
file4_path = os.path.join('C:/Users/salma/Downloads/file4') # CHANGE
if not os.path.exists(file4_path):
    os.makedirs(file4_path)
    logging.info(f"Folder 'file4' created at {file4_path}")

In [59]:
## Main 2: Itearte Through files and run Image Sequence, GrayScaling, ATD and Image Stacking


# Iterate through the directory and its subdirectories
first_iteration = True
for root, dirs, files in os.walk(base_directory):
    
    # Skip the first iteration
    if first_iteration:
        first_iteration = False
        continue  
    
    # Create folder for num of words (File2 - image sequence)
    root_image_path = os.path.join(file2_path, os.path.relpath(root, base_directory))
    if not os.path.exists(root_image_path):
        os.makedirs(root_image_path)
        
    # Create folder for num of words (File3 - grayscale images)
    root_grayscale_path = os.path.join(file3_path, os.path.relpath(root, base_directory))
    if not os.path.exists(root_grayscale_path):
        os.makedirs(root_grayscale_path)
        
    # Create folder for num of words (File4 - ATD and stacked images)
    root_atd_path = os.path.join(file4_path, os.path.relpath(root, base_directory))
    if not os.path.exists(root_atd_path):
        os.makedirs(root_atd_path)
    
    for directory in dirs:
        # Create folder for participant (File2 - image sequence)
        par_image_path = os.path.join(root_image_path, directory)
        if not os.path.exists(par_image_path):
            os.makedirs(par_image_path)
            
        # Create folder for participant (File3 - grayscale image)
        par_grayimage_path = os.path.join(root_grayscale_path, directory)
        if not os.path.exists(par_grayimage_path):
            os.makedirs(par_grayimage_path)
            
        # Create folder for participant (File4 - ATD and stacked images)
        par_atd_path = os.path.join(root_atd_path, directory)
        if not os.path.exists(par_atd_path):
            os.makedirs(par_atd_path)
        
        # Run Image Sequence Code
        par_vid_path = os.path.join(root, directory)
        output_path = create_folders_and_export_videos(par_vid_path, par_image_path)
        
        # Run GrayScale
        output_gray_path = os.path.join(par_grayimage_path, os.path.relpath(output_path, par_image_path))
        if not os.path.exists(output_gray_path):
            os.makedirs(output_gray_path)
        grayImagesList = convertToGrayScale(output_path, output_gray_path)
        
        # Run ATD
        output_atd_path = os.path.join(par_atd_path, os.path.relpath(output_gray_path, par_grayimage_path))
        if not os.path.exists(output_atd_path):
            os.makedirs(output_atd_path)
        
        noFrames = len(grayImagesList)
        # n + 2
        noWords =  5 # ex: for 3 words
        split = noFrames // noWords
        
        for i in range(noWords): # Drop first and last split 
            index = split * i
            applyATD(grayImagesList[index: index + split], output_atd_path, i)
        

.svo file 'fAG_01.svo' exported and saved in 'C:/Users/salma/Downloads/file2\2 words\AG\fAG_01'
video images 'fAG_01' converted to grayscale and saved in 'C:/Users/salma/Downloads/file3\2 words\AG\fAG_01'


UnboundLocalError: cannot access local variable 'output_path' where it is not associated with a value